In [2]:
import torch
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Lista dei nomi dei file dei modelli
model_files = [f'pruned_model{i}.pth' for i in range(1, 11)]

# Lista dei livelli dei layer da analizzare
levels = [0, 2, 4, 6]

# Dizionario per accumulare i risultati
results = {level: {'num_zeri': [], 'perc_zeri': []} for level in levels}

for model_file in model_files:
    it1 = torch.load(model_file, map_location=device)
    
    for level in levels:
        weights = it1[f'classifier.{level}.weight']
        weights_df = pd.DataFrame(weights.numpy())

        num_zeri = (weights_df == 0).sum().sum()
        total_param = weights_df.size
        
        perc_zeri = (num_zeri / total_param) * 100

        results[level]['num_zeri'].append(num_zeri)
        results[level]['perc_zeri'].append(perc_zeri)

# Calcolare la media dei risultati per ogni livello
for level in levels:
    avg_num_zeri = sum(results[level]['num_zeri']) / len(results[level]['num_zeri'])
    avg_perc_zeri = sum(results[level]['perc_zeri']) / len(results[level]['perc_zeri'])
    
    print(f'lvl {level}: media num. collegamenti a 0: {avg_num_zeri}, media {avg_perc_zeri:.2f}% di zeri')


lvl 0: media num. collegamenti a 0: 368357.8, media 93.97% di zeri
lvl 2: media num. collegamenti a 0: 135606.4, media 90.40% di zeri
lvl 4: media num. collegamenti a 0: 24442.6, media 81.48% di zeri
lvl 6: media num. collegamenti a 0: 637.0, media 63.70% di zeri
